In [1]:
import radosgw

access_key = '6LDLPN722I69N3AY9B95'
secret_key = 'nyPiW6pGu9sOEWlCTexOgXu54UMvMBm1zGzqyv3W'

rgwadmin = radosgw.connection.RadosGWAdminConnection(host='192.168.35.41', 
                                                     is_secure=False,
                                                     access_key=access_key,
                                                     secret_key=secret_key)

testuser1 = rgwadmin.get_user('mania')

bucket_info = rgwadmin.get_bucket('data', stats=True)
for key in bucket_info._object:
    print(key, '\t', bucket_info._object[key])

bucket 	 data
zonegroup 	 56960513-3595-4337-9d8a-8b6b68cd76bb
placement_rule 	 default-placement
explicit_placement 	 {'data_pool': '', 'data_extra_pool': '', 'index_pool': ''}
id 	 3ba5b0d9-ab69-488c-909d-7cb0237cafba.9672886.2
marker 	 3ba5b0d9-ab69-488c-909d-7cb0237cafba.9286704.1
index_type 	 Normal
owner 	 testuser
ver 	 0#114243,1#114887,2#114514,3#114573,4#114163,5#114315
master_ver 	 0#0,1#0,2#0,3#0,4#0,5#0
mtime 	 2020-05-18 11:38:30.271801
max_marker 	 0#,1#,2#,3#,4#,5#
usage 	 {'rgw.main': {'size': 1838756186444, 'size_actual': 1839720906752, 'size_utilized': 1838756186444, 'size_kb': 1795660339, 'size_kb_actual': 1796602448, 'size_kb_utilized': 1795660339, 'num_objects': 508428}, 'rgw.multimeta': {'size': 0, 'size_actual': 0, 'size_utilized': 0, 'size_kb': 0, 'size_kb_actual': 0, 'size_kb_utilized': 0, 'num_objects': 19}}
bucket_quota 	 {'enabled': False, 'check_on_raw': True, 'max_size': -1, 'max_size_kb': 0, 'max_objects': -1}


In [2]:
import swiftclient
import json 

user = 'testuser:swift'
key = '7Xqb6gdsCE5Vu0clmk2qL0yjjy1NCNiFuaPlGQvJ'



def add_to_metadata_tree(metadata, name, size):
    path_element = name.split('/')
    
    meta_ptr = metadata
    for element in path_element:
        if element not in meta_ptr:
            meta_ptr[element] = {'objs': {}, 'size': 0 }
        meta_ptr[element]['size'] += size
        meta_ptr = meta_ptr[element]['objs']

def build_metadata_tree():    
    conn = swiftclient.Connection(
            user=user,
            key=key,
            authurl='http://192.168.35.41:80/auth/1.0')

    bucket_name = 'data'    
    metadata = {}
    
    metadata_swift =  conn.get_container(bucket_name)[1]
    for data in metadata_swift:
        full_name = data['name'];

        if full_name.endswith('_SUCCESS') : continue;
        add_to_metadata_tree(metadata, data['name'], data['bytes'])
        #break
        print(data['name'], data['bytes'])
    
build_metadata_tree()
#print(json.dumps(metadata, indent = 4))


HiBench/Aggregation/Input/rankings/part-00000 27758205
HiBench/Aggregation/Input/rankings/part-00001 27756809
HiBench/Aggregation/Input/rankings/part-00002 27755614
HiBench/Aggregation/Input/rankings/part-00003 27751598
HiBench/Aggregation/Input/rankings/part-00004 27729477
HiBench/Aggregation/Input/rankings/part-00005 27746656
HiBench/Aggregation/Input/rankings/part-00006 27768171
HiBench/Aggregation/Input/rankings/part-00007 27776797
HiBench/Aggregation/Input/rankings/part-00008 27772786
HiBench/Aggregation/Input/rankings/part-00009 27751651
HiBench/Aggregation/Input/rankings/part-00010 27746082
HiBench/Aggregation/Input/rankings/part-00011 27759893
HiBench/Aggregation/Input/rankings/part-00012 27742596
HiBench/Aggregation/Input/rankings/part-00013 27764949
HiBench/Aggregation/Input/rankings/part-00014 27755532
HiBench/Aggregation/Input/rankings/part-00015 27761174
HiBench/Aggregation/Input/rankings/part-00016 27756707
HiBench/Aggregation/Input/rankings/part-00017 27742288
HiBench/Ag

pig-tpch/100G/customer/customer.tbl.12 12149716
pig-tpch/100G/customer/customer.tbl.120 12218475
pig-tpch/100G/customer/customer.tbl.121 12209930
pig-tpch/100G/customer/customer.tbl.122 12230672
pig-tpch/100G/customer/customer.tbl.123 12241239
pig-tpch/100G/customer/customer.tbl.124 12218119
pig-tpch/100G/customer/customer.tbl.125 12234838
pig-tpch/100G/customer/customer.tbl.126 12221312
pig-tpch/100G/customer/customer.tbl.127 12221800
pig-tpch/100G/customer/customer.tbl.128 12218378
pig-tpch/100G/customer/customer.tbl.129 12232923
pig-tpch/100G/customer/customer.tbl.13 12145420
pig-tpch/100G/customer/customer.tbl.130 12228206
pig-tpch/100G/customer/customer.tbl.131 12224757
pig-tpch/100G/customer/customer.tbl.132 12222172
pig-tpch/100G/customer/customer.tbl.133 12213536
pig-tpch/100G/customer/customer.tbl.134 12278400
pig-tpch/100G/customer/customer.tbl.135 12298576
pig-tpch/100G/customer/customer.tbl.136 12304058
pig-tpch/100G/customer/customer.tbl.137 12305358
pig-tpch/100G/customer

pig-tpch/100G/part/part.tbl.49 12157970
pig-tpch/100G/part/part.tbl.5 12058057
pig-tpch/100G/part/part.tbl.50 12160735
pig-tpch/100G/part/part.tbl.51 12155927
pig-tpch/100G/part/part.tbl.52 12163763
pig-tpch/100G/part/part.tbl.53 12151920
pig-tpch/100G/part/part.tbl.54 12160752
pig-tpch/100G/part/part.tbl.55 12156347
pig-tpch/100G/part/part.tbl.56 12160713
pig-tpch/100G/part/part.tbl.57 12152714
pig-tpch/100G/part/part.tbl.58 12160320
pig-tpch/100G/part/part.tbl.59 12156703
pig-tpch/100G/part/part.tbl.6 12064070
pig-tpch/100G/part/part.tbl.60 12158871
pig-tpch/100G/part/part.tbl.61 12153782
pig-tpch/100G/part/part.tbl.62 12156532
pig-tpch/100G/part/part.tbl.63 12157743
pig-tpch/100G/part/part.tbl.64 12159948
pig-tpch/100G/part/part.tbl.65 12153394
pig-tpch/100G/part/part.tbl.66 12160336
pig-tpch/100G/part/part.tbl.67 12153819
pig-tpch/100G/part/part.tbl.68 12160921
pig-tpch/100G/part/part.tbl.69 12156656
pig-tpch/100G/part/part.tbl.7 12058707
pig-tpch/100G/part/part.tbl.70 12160943
pig

pig-tpch/32G/customer/customer.tbl.21 12227384
pig-tpch/32G/customer/customer.tbl.22 12230880
pig-tpch/32G/customer/customer.tbl.23 12219393
pig-tpch/32G/customer/customer.tbl.24 12223338
pig-tpch/32G/customer/customer.tbl.25 12213406
pig-tpch/32G/customer/customer.tbl.26 12226993
pig-tpch/32G/customer/customer.tbl.27 12227703
pig-tpch/32G/customer/customer.tbl.28 12217451
pig-tpch/32G/customer/customer.tbl.29 12209175
pig-tpch/32G/customer/customer.tbl.3 12153475
pig-tpch/32G/customer/customer.tbl.30 12230703
pig-tpch/32G/customer/customer.tbl.31 12221712
pig-tpch/32G/customer/customer.tbl.32 12220993
pig-tpch/32G/customer/customer.tbl.33 12217332
pig-tpch/32G/customer/customer.tbl.34 12222138
pig-tpch/32G/customer/customer.tbl.35 12224086
pig-tpch/32G/customer/customer.tbl.36 12224596
pig-tpch/32G/customer/customer.tbl.37 12229297
pig-tpch/32G/customer/customer.tbl.38 12224111
pig-tpch/32G/customer/customer.tbl.39 12229489
pig-tpch/32G/customer/customer.tbl.4 12153501
pig-tpch/32G/cu

pig-tpch/64G/orders/orders.tbl.63 88082367
pig-tpch/64G/orders/orders.tbl.64 88096832
pig-tpch/64G/orders/orders.tbl.65 88094825
pig-tpch/64G/orders/orders.tbl.66 88088255
pig-tpch/64G/orders/orders.tbl.67 88065671
pig-tpch/64G/orders/orders.tbl.68 88106463
pig-tpch/64G/orders/orders.tbl.69 88085891
pig-tpch/64G/orders/orders.tbl.7 87352908
pig-tpch/64G/orders/orders.tbl.70 88089581
pig-tpch/64G/orders/orders.tbl.71 88064150
pig-tpch/64G/orders/orders.tbl.72 88034285
pig-tpch/64G/orders/orders.tbl.73 88084187
pig-tpch/64G/orders/orders.tbl.74 88085704
pig-tpch/64G/orders/orders.tbl.75 88081940
pig-tpch/64G/orders/orders.tbl.76 88041856
pig-tpch/64G/orders/orders.tbl.77 88068710
pig-tpch/64G/orders/orders.tbl.78 88085473
pig-tpch/64G/orders/orders.tbl.79 88045288
pig-tpch/64G/orders/orders.tbl.8 87313669
pig-tpch/64G/orders/orders.tbl.80 88096875
pig-tpch/64G/orders/orders.tbl.81 88071399
pig-tpch/64G/orders/orders.tbl.82 88098660
pig-tpch/64G/orders/orders.tbl.83 88064937
pig-tpch/64G/

...

tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452312/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 81438
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452313/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 78002
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452314/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 80034
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452315/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 76818
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452316/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 78032
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452317/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 76667
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452318/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 82226
tpcds-par-2g/catalog_sales/cs_sold_date_sk=2452319/part-00000-a6ba736b-e13f-42fa-b4fb-52104fca6115.c000.snappy.parquet 79948


tpcds-par-2g/inventory/inv_date_sk=2452530/part-00093-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 121752
tpcds-par-2g/inventory/inv_date_sk=2452530/part-00094-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 187121
tpcds-par-2g/inventory/inv_date_sk=2452537/part-00094-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 254009
tpcds-par-2g/inventory/inv_date_sk=2452544/part-00094-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 244145
tpcds-par-2g/inventory/inv_date_sk=2452544/part-00095-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 22126
tpcds-par-2g/inventory/inv_date_sk=2452551/part-00095-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 253995
tpcds-par-2g/inventory/inv_date_sk=2452558/part-00095-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 254042
tpcds-par-2g/inventory/inv_date_sk=2452565/part-00095-05319124-9133-4374-a73c-25b12be62f79.c000.snappy.parquet 167423
tpcds-par-2g/inventory/inv_date_sk=2452565/part-00096-053

tpcds-par-2g/store_returns/sr_returned_date_sk=2451497/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 25846
tpcds-par-2g/store_returns/sr_returned_date_sk=2451498/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 28515
tpcds-par-2g/store_returns/sr_returned_date_sk=2451499/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 25173
tpcds-par-2g/store_returns/sr_returned_date_sk=2451500/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 26478
tpcds-par-2g/store_returns/sr_returned_date_sk=2451501/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 27650
tpcds-par-2g/store_returns/sr_returned_date_sk=2451502/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 28815
tpcds-par-2g/store_returns/sr_returned_date_sk=2451503/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 28562
tpcds-par-2g/store_returns/sr_returned_date_sk=2451504/part-00000-68518dc9-f8b1-4ff0-b3a5-8910fac

tpcds-par-2g/store_returns/sr_returned_date_sk=2452193/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 23636
tpcds-par-2g/store_returns/sr_returned_date_sk=2452194/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 21974
tpcds-par-2g/store_returns/sr_returned_date_sk=2452195/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 23662
tpcds-par-2g/store_returns/sr_returned_date_sk=2452196/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 23648
tpcds-par-2g/store_returns/sr_returned_date_sk=2452197/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 25785
tpcds-par-2g/store_returns/sr_returned_date_sk=2452198/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 24668
tpcds-par-2g/store_returns/sr_returned_date_sk=2452199/part-00000-68518dc9-f8b1-4ff0-b3a5-8910facb24fc.c000.snappy.parquet 24829
tpcds-par-2g/store_returns/sr_returned_date_sk=2452200/part-00000-68518dc9-f8b1-4ff0-b3a5-8910fac

tpcds-par-2g/store_sales/ss_sold_date_sk=2450878/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 99916
tpcds-par-2g/store_sales/ss_sold_date_sk=2450879/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 109813
tpcds-par-2g/store_sales/ss_sold_date_sk=2450880/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 115570
tpcds-par-2g/store_sales/ss_sold_date_sk=2450881/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 96392
tpcds-par-2g/store_sales/ss_sold_date_sk=2450882/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 101556
tpcds-par-2g/store_sales/ss_sold_date_sk=2450883/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 96608
tpcds-par-2g/store_sales/ss_sold_date_sk=2450884/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 88390
tpcds-par-2g/store_sales/ss_sold_date_sk=2450885/part-00000-7793c2e1-30eb-4d55-8113-de1b4cda1407.c000.snappy.parquet 99962
tpcds-par-2g/

In [158]:
import math
import requests
import swiftclient
import json 


user = 'testuser:swift'
key = '7Xqb6gdsCE5Vu0clmk2qL0yjjy1NCNiFuaPlGQvJ'


def fetch_object(token, bucket_name, obj_name, ofs_s, ofs_e):
    print(bucket_name, obj_name)
    
    url = 'http://192.168.35.41:80/swift/v1/%s/%s'%(bucket_name, obj_name)
    headers = {"range":"bytes=%d-%d"%(ofs_s, ofs_e),
              "X-Auth-Token": token}
    
    print(url, headers)
    
    
    r=requests.get(url, headers=headers)
    
    print(len(r.content), ofs_e - ofs_s)
    '''
    local_p = subprocess.Popen(["curl -i http://%s:%d/swift/v1/%s/%s -X GET -H \"range: bytes=%d-%d\" -H \"KARIZ_PREFETCH: 1\" -H \"X-Auth-Token: %s\" -o /dev/null 2>&1 | tee %soutput-%s.txt" %(cfg.d3n_host, cfg.d3n_port, bucket_name, obj_name, s_off, e_off, token, './', obj_name)], shell=True)
    local_p.wait()
    '''



def prefetch_dataset(path, wave=-1, stride=0):
    # stride = -1 means prefetch the full object
    # wave = -1 means follow the partitions schema of original dataset
    cache_block_size = 4194304 # 4 MB
    
    path_element = path.split('/')
    
    meta_ptr = metadata
    for element in path_element:
        if element not in meta_ptr:
            return -1; # means could not prefetch this input
        meta_ptr = meta_ptr[element]['objs']
        
    for obj in meta_ptr:
        n_cache_blocks = meta_ptr[obj]['size']//cache_block_size
        
            
        ofs_s = (n_cache_blocks - stride)*cache_block_size if (stride) else 0 
        ofs_e = meta_ptr[obj]['size']
        
        fetch_object(token=token, bucket_name='data', obj_name=path+'/'+obj, ofs_s = ofs_s, ofs_e = ofs_e)
        
        # add to thread pool 
        
        
    
    #print(json.dumps(meta_ptr, indent = 4))

conn = swiftclient.Connection(
        user=user,
        key=key,
        authurl='http://192.168.35.41:80/auth/1.0')

token = conn.get_auth()[1]
print(token)
    
dpath = 'HiBench/Wordcount/Input'
prefetch_dataset(dpath, stride=176)

AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a
data HiBench/Wordcount/Input/part-m-00000
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00000 {'range': 'bytes=360710144-1102234211', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741524067 741524067
data HiBench/Wordcount/Input/part-m-00001
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00001 {'range': 'bytes=360710144-1102230641', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741520497 741520497
data HiBench/Wordcount/Input/part-m-00002
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00002 {'range': 'bytes=360710144-1102224903', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e1579032

741522071 741522071
data HiBench/Wordcount/Input/part-m-00026
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00026 {'range': 'bytes=360710144-1102245692', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741535548 741535548
data HiBench/Wordcount/Input/part-m-00027
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00027 {'range': 'bytes=360710144-1102239104', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741528960 741528960
data HiBench/Wordcount/Input/part-m-00028
http://192.168.35.41:80/swift/v1/data/HiBench/Wordcount/Input/part-m-00028 {'range': 'bytes=360710144-1102241471', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667451f8fe976a89ede7ec28aa5e15790324102b81d9602004ff9de5c245fce8f3159e65b20a'}
741531327 741531327
data HiBench/Wordcount/Input/part-m

In [173]:
token = conn.get_auth()[1]
print(token)

def clear_cache():
    url = 'http://192.168.35.41:80/swift/v1/'
    headers = {"KARIZ_FLUSH_CACHE":"1",
              "X-Auth-Token": token}

    r=requests.delete(url, headers=headers)
    print(r)



#curl -i http://192.168.35.41:80/swift/v1/ -X DELETE -H "KARIZ_FLUSH_CACHE" -H "X-Auth-Token: $OS_AUTH_TOKEN"

AUTH_rgwtk0e00000074657374757365723a737769667401fa2c695db584693509ab5e1c8647367ad0222e648226b8451c455263b0451fa58d3b66
http://192.168.35.41:80/swift/v1/ {'KARIZ_FLUSH_CACHE': '1', 'X-Auth-Token': 'AUTH_rgwtk0e00000074657374757365723a737769667401fa2c695db584693509ab5e1c8647367ad0222e648226b8451c455263b0451fa58d3b66'}
<Response [200]>


In [183]:
oids0 = '/local0/Kariz/expriments/microbenchmark/oids0'
oids1 = '/local0/Kariz/expriments/microbenchmark/oids1'

oids0_dict = {}
found = 0

with open(oids0, 'r') as fd0, open(oids1, 'r') as fd1:
    lns0 = fd0.readlines()
    lns1 = fd1.readlines()
    
    lns0 = [ln.split('__shadow_.')[1] for ln in lns0]
    lns1 = [ln.split('__shadow_.')[1] for ln in lns1]
    
    for oid in lns0:
        oids0_dict[oid] = 1;
        
    for oid in lns1:
        if oid in oids0_dict:
            found += 1;
    
    print(found, len(lns0), len(lns1), lns0[0], lns1[0])

0 8300 4096 44ZmSgPSdssfWV5nZAIiknKEH7mn0kS_1
 _3OPiXJZXMgLg-kubf3Pdbt5aohV4Yj_134

